In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U bitsandbytes  
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install -q -U transformers accelerate peft

In [5]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN2")
login(token=hf_token)

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
%%time
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# 1. Secure Authentication
user_secrets = UserSecretsClient()
try:
    hf_token = user_secrets.get_secret("HF_TOKEN2")
    login(token=hf_token)
    print("✅ Login Successful")
except:
    print("❌ Error: HF_TOKEN not found in Kaggle Secrets.")

# 2. Resource Management Logic
model_id = "google/medgemma-1.5-4b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🚀 Initializing model on: {device.upper()}")

# 3. Load Model (Optimized for CPU/GPU)
processor = AutoProcessor.from_pretrained(model_id)

if device == "cuda":
    # GPU Mode: Fast 4-bit quantization
    model = AutoModelForImageTextToText.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        load_in_4bit=True
    )
else:
    # CPU Mode: Save your 30hr quota by using full RAM
    print("ℹ️ GPU not detected. Running in CPU mode (Debugging only).")
    model = AutoModelForImageTextToText.from_pretrained(
        model_id,
        torch_dtype=torch.float32,
        device_map="cpu"
    )

print(f"🎉 Model Loaded Successfully on {device}!")

2026-01-15 21:36:15.003354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768512975.209601      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768512975.270728      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768512975.772394      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768512975.772449      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768512975.772452      55 computation_placer.cc:177] computation placer alr

✅ Login Successful
🚀 Initializing model on: CUDA


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

🎉 Model Loaded Successfully on cuda!
CPU times: user 1min 13s, sys: 42.8 s, total: 1min 55s
Wall time: 1min 28s


In [9]:
%%time
import torch
from transformers import pipeline
from PIL import Image
import requests
from io import BytesIO

# 1. Update your image loading logic
url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

pipe = pipeline(
    "image-text-to-text", 
    model="google/medgemma-1.5-4b-it", 
    torch_dtype=torch.bfloat16, 
    device_map="auto",
    model_kwargs={"load_in_4bit": True} # Save VRAM for large datasets
)


response = requests.get(url, headers=headers)
if response.status_code == 200:
    # Use BytesIO to handle the raw content safely
    image = Image.open(BytesIO(response.content)).convert("RGB")
    print("✅ Image loaded and converted to RGB successfully.")
else:
    print(f"❌ Failed to download image. Status code: {response.status_code}")

# 2. Run your inference again
# (Using the prompt and messages from our previous step)
prompt = """Analyze this chest X-ray as a senior radiologist. 
1. Identify key anatomical landmarks.
2. Note any abnormal opacities or findings.
3. Provide a summary of the clinical impression."""

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ],
    },
]

# Run the pipeline (ensure 'pipe' is defined from our previous step)
output = pipe(messages, max_new_tokens=512)
print("-" * 30)
print(output[0]['generated_text'])

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


✅ Image loaded and converted to RGB successfully.
------------------------------
[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=2412x1956 at 0x7A18B66202C0>}, {'type': 'text', 'text': 'Analyze this chest X-ray as a senior radiologist. \n1. Identify key anatomical landmarks.\n2. Note any abnormal opacities or findings.\n3. Provide a summary of the clinical impression.'}]}, {'role': 'assistant', 'content': "Here's an analysis of the provided chest X-ray:\n\n1.  **Anatomical Landmarks:**\n    *   **Heart:** Appears normal in size and contour.\n    *   **Mediastinum:** Normal in width and appearance.\n    *   **Ribs:** Visible and appear intact.\n    *   **Clavicles:** Visible.\n    *   **Diaphragm:** The right hemidiaphragm is visible.\n    *   **Lungs:** The lung fields are clear with good visualization of the lung markings.\n\n2.  **Abnormalities:**\n    *   **No significant abnormalities** are immediately apparent on this single image.\n\n

In [10]:
%%time
import os
from PIL import Image

# 1. Path to one of the JPGs in your added dataset
test_image_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/yolov8_roi_detection/experiment/val_batch0_pred.jpg'

if os.path.exists(test_image_path):
    # 2. Load the image
    test_img = Image.open(test_image_path).convert("RGB")
    print(f"✅ Loaded test image: {test_image_path}")

    # 3. MedGemma Prompt
    # Since these are YOLO predictions, we can ask the model to describe the boxes it sees.
    prompt = "This is a CT slice with YOLO detection boxes. Describe the organs identified and check for any signs of injury inside the boxes."

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": test_img},
                {"type": "text", "text": prompt},
            ],
        },
    ]

    # 4. Inference
    print("🚀 Testing MedGemma reasoning...")
    output = pipe(messages, max_new_tokens=256)
    print("\n🩺 ANALYSIS:\n", output[0]['generated_text'])
else:
    print("❌ Still can't find that image. Let's list the input directory to be sure.")
    print(os.listdir('/kaggle/input/'))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


✅ Loaded test image: /kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/yolov8_roi_detection/experiment/val_batch0_pred.jpg
🚀 Testing MedGemma reasoning...

🩺 ANALYSIS:
 [{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1920x1808 at 0x7A18B67CB4D0>}, {'type': 'text', 'text': 'This is a CT slice with YOLO detection boxes. Describe the organs identified and check for any signs of injury inside the boxes.'}]}, {'role': 'assistant', 'content': "This is a CT slice image showing anatomical structures, likely from a veterinary or veterinary medicine context, based on the labels. The image displays several regions and organs.\n\nHere's a breakdown of the identified organs and potential observations:\n\n**Abdominal and Thoracic Regions:**\n*   **Abdominal:** The image shows the abdomen region, with several structures visible, including intestines, the liver, and the kidneys.\n*   **Lower:** This likely refers to the lower portion of the image,

In [11]:
# Updated Clinical Prompt
clinical_prompt = """You are a senior trauma radiologist assistant. 
Review the bounding boxes in this CT scan specifically for:
1. Lacerations or hematomas in the Liver/Spleen.
2. Active extravasation (bright contrast pooling).
3. Free fluid in the peritoneal cavity.

Identify the findings in the highlighted ROIs and categorize them by organ."""

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": test_img},
            {"type": "text", "text": clinical_prompt},
        ],
    },
]

# Run it again
output = pipe(messages, max_new_tokens=512)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1920x1808 at 0x7A18B67CB4D0>}, {'type': 'text', 'text': 'You are a senior trauma radiologist assistant. \nReview the bounding boxes in this CT scan specifically for:\n1. Lacerations or hematomas in the Liver/Spleen.\n2. Active extravasation (bright contrast pooling).\n3. Free fluid in the peritoneal cavity.\n\nIdentify the findings in the highlighted ROIs and categorize them by organ.'}]}, {'role': 'assistant', 'content': 'FINDINGS:\n\n*   **Liver/Spleen**: No obvious lacerations or hematomas are visualized in the liver or spleen based on the provided images.\n*   **Abdomen**: Free fluid in the peritoneal cavity is visualized.\n*   **Other**: The images show areas of interest within the abdominal cavity.\n\nBased on the provided images, the following findings are identified:\n\n*   **Free fluid**: Free fluid is present in the peritoneal cavity, indicated by the bounding boxes highlighted in th

In [12]:
%%time
import pandas as pd
import os

# 1. Load the labels (Adjust path to your folds/train csv)
train_df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/folds.csv')

def get_ground_truth(image_path):
    # Extract patient_id from the path
    # Path format: /kaggle/.../patient_id/series_id/filename
    parts = image_path.split('/')
    
    # Adjust index depending on your specific folder structure
    # Usually patient_id is the 2nd or 3rd to last folder
    try:
        patient_id = int(parts[-3]) 
        labels = train_df[train_df['patient_id'] == patient_id].iloc[0]
        
        print(f"📌 GROUND TRUTH FOR PATIENT {patient_id}:")
        print(f"Liver: {'Healthy' if labels.liver_healthy else 'Injured (Low/High)'}")
        print(f"Spleen: {'Healthy' if labels.spleen_healthy else 'Injured (Low/High)'}")
        print(f"Kidney: {'Healthy' if labels.kidney_healthy else 'Injured (Low/High)'}")
        return labels
    except Exception as e:
        return f"Could not parse patient ID from path: {e}"

# Run it on your test image
gt_labels = get_ground_truth("/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/yolov8_roi_detection/experiment/val_batch0_pred.jpg")

CPU times: user 6.37 ms, sys: 1.18 ms, total: 7.55 ms
Wall time: 14.9 ms


In [13]:
%%time
import pandas as pd
import os
import glob

# 1. Load the Baseline Scores
scores_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/baseline/scores.csv'
scores_df = pd.read_csv(scores_path)

# 2. Find any image file in your dataset to test the link
def find_all_images(root_dir):
    # Support jpg, png, and dcm
    valid_extensions = ('.jpg', '.png', '.dcm')
    image_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(valid_extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

# 3. Map Image to Score
all_imgs = find_all_images('/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/')

print(f"📂 Found {len(all_imgs)} images in total.")

if all_imgs:
    sample_path = all_imgs[0]
    print(f"🔍 Testing mapping for: {sample_path}")
    
    # Extract ID (This is the 'Startup' way: iterate until it hits)
    # Most Kaggle datasets use the filename or the parent folder as the ID
    potential_id = os.path.basename(sample_path).split('.')[0].split('_')[-1]
    
    # Look for this ID in your scores_df
    # We search the whole dataframe for any column containing this ID
    match = scores_df[scores_df.astype(str).apply(lambda x: x.str.contains(potential_id)).any(axis=1)]
    
    if not match.empty:
        print("✅ Match Found in Baseline Scores!")
        print(match.head())
    else:
        print(f"❌ ID {potential_id} not found in scores.csv. Let's look at the first few IDs in the CSV:")
        print(scores_df.head())

📂 Found 180 images in total.
🔍 Testing mapping for: /kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/yolov8_roi_detection/experiment/PR_curve.png
❌ ID curve not found in scores.csv. Let's look at the first few IDs in the CSV:
                 target  log_loss  sample_weighted_log_loss  accuracy  \
0          bowel_injury  0.105383                  0.167508  0.979663   
1  extravasation_injury  0.671142                  0.682697  0.936447   
2            any_injury  0.681792                  0.703257  0.728313   
3                kidney  0.299856                  0.511072  0.942167   
4                 liver  0.427419                  0.652512  0.897998   

   precision    recall  specificity        f1  roc_auc  average_precision  
0   0.000000  0.000000          1.0  0.000000      0.5           0.020337  
1   0.000000  0.000000          1.0  0.000000      0.5           0.063553  
2   0.000000  0.000000          1.0  0.000000      0.5           0.271687  
3   0.314056  0.33333

In [14]:
%%time
import os
from PIL import Image

# 1. Filter for actual CT slice images (avoiding the .png curves)
all_files = find_all_images('/kaggle/input/rsna-2023-abdominal-trauma-detection-dataset/')
# We want the 'val_batch' images because they contain the actual patient data
scan_images = [f for f in all_files if 'val_batch' in f and f.endswith('.jpg')]

print(f"✅ Found {len(scan_images)} patient scan samples for analysis.")

# 2. Clinical Report Generator Loop
for i, img_path in enumerate(scan_images[:3]): # Let's start with 3 samples
    print(f"\n📄 GENERATING REPORT FOR SAMPLE {i+1}: {os.path.basename(img_path)}")
    
    img = Image.open(img_path).convert("RGB")
    
    # Advanced Clinical Prompt: Forcing the model into 'Radiologist' mode
    report_prompt = """Perform a formal radiological review of this abdominal CT slice.
    - Assessment: Identify organs within the YOLO bounding boxes.
    - Findings: Describe any density changes, lacerations, or fluid.
    - Conclusion: Based on the visual evidence, what is the suspected injury grade?"""

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": report_prompt},
            ],
        },
    ]

    # Inference
    output = pipe(messages, max_new_tokens=350)
    
    print("-" * 40)
    print(f"🩺 MEDGEMMA REPORT {i+1}:")
    print(output[0]['generated_text'])
    print("-" * 40)

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


✅ Found 6 patient scan samples for analysis.

📄 GENERATING REPORT FOR SAMPLE 1: val_batch0_pred.jpg


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


----------------------------------------
🩺 MEDGEMMA REPORT 1:
[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1920x1808 at 0x7A18B662C8F0>}, {'type': 'text', 'text': 'Perform a formal radiological review of this abdominal CT slice.\n    - Assessment: Identify organs within the YOLO bounding boxes.\n    - Findings: Describe any density changes, lacerations, or fluid.\n    - Conclusion: Based on the visual evidence, what is the suspected injury grade?'}]}, {'role': 'assistant', 'content': '- The image displays a series of CT slices focused on the abdomen.\n- The bounding boxes highlight areas of interest.\n- In the first image, the "abdominal 1.0" box highlights the stomach and the surrounding abdominal contents.\n- In the second image, the "lower 1.0" box highlights the abdominal contents, with the lower portion of the stomach and surrounding structures.\n- In the third image, the "lower 1.0" box highlights the abdominal contents, with the l

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


----------------------------------------
🩺 MEDGEMMA REPORT 2:
[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1920x1808 at 0x7A18B4237440>}, {'type': 'text', 'text': 'Perform a formal radiological review of this abdominal CT slice.\n    - Assessment: Identify organs within the YOLO bounding boxes.\n    - Findings: Describe any density changes, lacerations, or fluid.\n    - Conclusion: Based on the visual evidence, what is the suspected injury grade?'}]}, {'role': 'assistant', 'content': 'I am unable to perform a formal radiological review of the provided image. I am unable to process images.\n\nHowever, I can provide a general assessment based on the name "abdominal CT slice" and the labels.\n\n- **Assessment:** The image is a CT scan of the abdomen. The YOLO bounding boxes likely highlight various anatomical structures.\n\n- **Findings:** Without a detailed description of the image, it is hard to find specific abnormalities.\n\n- **Conclus